In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
!pip install tensorflow_addons
import tensorflow_addons as tfa
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")    

from tensorflow.python.keras.utils.data_utils import Sequence

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


sklearn version:  1.0.2
TF version:  2.8.0
GPU installed:  True
4 Physical GPUs, 4 Logical GPUs


2022-06-08 02:45:24.876406: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-08 02:45:26.506903: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9530 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:19:00.0, compute capability: 7.5
2022-06-08 02:45:26.507948: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14516 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:1a:00.0, compute capability: 7.5
2022-06-08 02:45:26.508786: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GP

In [2]:
X_train=np.uint8(np.load('./X_train_aug.npy'))
y_train=np.uint8(np.load('./y_train_aug.npy'))
X_valid=np.uint8(np.load('./X_valid.npy'))
y_valid=np.uint8(np.load('./y_valid.npy'))

In [3]:
X_valid=tf.reshape(X_valid,[-1,28,28,1])
X_valid_resize=np.uint8(tf.image.resize(X_valid, [32, 32]))
X_train.shape,X_valid_resize.shape,X_train.dtype

2022-06-08 02:45:30.726828: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 671805440 exceeds 10% of free system memory.


((1074263, 32, 32, 1), (164015, 32, 32, 1), dtype('uint8'))

In [4]:
#황성현
class Inveted_Residual_Block(keras.layers.Layer):
    def __init__(self,c,s=1,t=1,n=1,**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.layers.ReLU(max_value=6)
        self.main_layers = [
            
            ]
        self.strides_layers=[
           ]
        self.s=s
        self.t=t
        self.c=c
        self.n=n
        
    def build(self,batch_input_shape):
        print(self.name,": batch_input_shape =",batch_input_shape)
        
        self.main_layers = [
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1, 
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=1,padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c,kernel_size=1, strides=1,padding="SAME", activation=self.activation),
            ]
        self.strides_layers=[
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1,
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=self.s,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c, kernel_size=1,strides=1,padding="SAME",activation=self.activation),
        ]

    def call(self, inputs):
        Z = inputs
        
        for n in range(0,self.n):
            if n == 0:
                for layer in self.strides_layers:
                    Z = layer(Z)
            else:
                skip_Z = Z
                for layer in self.main_layers:
                    Z = layer(Z)
                
                Z += skip_Z
    
        return Z
    
    def get_config(self):
        base=super().get_config()
        return{**base,"s":self.s,"t":self.t,"c":self.c,"n":self.n}

In [5]:
    #황성현 전민재
    #multygpu


    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    opt = tf.keras.optimizers.Adam(learning_rate=0.0003)
    batch_size=128
    smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 32, 32, 1)        4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)     

In [6]:
#전민재
class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
            rate = max(rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.lr, rate)

In [7]:
#황성현 전민재
from datetime import datetime
K = keras.backend
logs = "logs/" + datetime.now().strftime("SMV2L_lrsch_onecycle_aug_dropout_128_epoch40%Y%m%d-%H%M%S")
onecycle = OneCycleScheduler(len(X_train) // batch_size * 40, max_rate=0.003)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_best_lrsch_onecycle_aug_dropout_128_epoch40.h5",save_best_only=True)

history=smv2_l.fit(X_train,y_train,validation_data=(X_valid_resize,y_valid),batch_size = batch_size, epochs=40,callbacks=[tboard_callback, check_best_cb,onecycle])

2022-06-08 02:45:32.204779: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-08 02:45:32.204813: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-06-08 02:45:32.204870: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1630] Profiler found 4 GPUs
2022-06-08 02:45:32.205594: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-06-08 02:45:32.903906: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-08 02:45:32.904126: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-06-08 02:45:32.987746: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1100045312 exceeds 10% of free system memor

Epoch 1/40


2022-06-08 02:45:36.815273: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8204


 499/8393 [>.............................] - ETA: 3:46 - loss: 4.4853 - accuracy: 0.0756

2022-06-08 02:45:52.672136: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-08 02:45:52.672195: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.


...

8393/8393 [==============================] - ETA: 0s - loss: 1.2610 - accuracy: 0.6409

2022-06-08 02:49:56.631157: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 251658240 exceeds 10% of free system memory.


8393/8393 [==============================] - 263s 31ms/step - loss: 1.2610 - accuracy: 0.6409 - val_loss: 0.4642 - val_accuracy: 0.8328
Epoch 2/40
8392/8393 [============================>.] - ETA: 0s - loss: 0.4855 - accuracy: 0.8261

2022-06-08 02:54:13.023008: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 251658240 exceeds 10% of free system memory.


8393/8393 [==============================] - 256s 31ms/step - loss: 0.4855 - accuracy: 0.8261 - val_loss: 0.4240 - val_accuracy: 0.8448
Epoch 3/40
8393/8393 [==============================] - 256s 30ms/step - loss: 0.4388 - accuracy: 0.8404 - val_loss: 0.4393 - val_accuracy: 0.8394
Epoch 4/40
8393/8393 [==============================] - 255s 30ms/step - loss: 0.4190 - accuracy: 0.8467 - val_loss: 0.4195 - val_accuracy: 0.8451
Epoch 5/40
8393/8393 [==============================] - 256s 30ms/step - loss: 0.4065 - accuracy: 0.8502 - val_loss: 0.3963 - val_accuracy: 0.8528
Epoch 6/40
8393/8393 [==============================] - 256s 31ms/step - loss: 0.3977 - accuracy: 0.8532 - val_loss: 0.4928 - val_accuracy: 0.8264
Epoch 7/40
8393/8393 [==============================] - 255s 30ms/step - loss: 0.3925 - accuracy: 0.8547 - val_loss: 0.5907 - val_accuracy: 0.8090
Epoch 8/40
8393/8393 [==============================] - 255s 30ms/step - loss: 0.3893 - accuracy: 0.8557 - val_loss: 0.4053 - val

In [31]:
smv2_l.save('./smv2l_adam_lrsch_onecycle_aug_dropout_128_epoch40.h5')